# Descripción

El ejercicio 1 se basa en descargar el archivo ***all_data.csv*** que es la Base de datos histórica de *Quién es Quién en los Precios* de Profeco.
El cuál se basa en cinco preguntas:
<ol>
  <li>¿Cuántos registros hay?</li>
  <li>¿Cuántas categorías?</li>
  <li>¿Cuántas cadenas comerciales están siendo monitoreadas (y, por lo tanto, reportadas en esa base de datos)?</li>
    <li>¿Cuáles son los productos más monitoreados en cada estado de la república?</li>
    <li>¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?<l/i>
</ol>

Las herramientas a usar son jupyter y pyspark  en su versión 2.4.5.

## Introducción

Se realiza la  importaciones de los módulos para trabajar con el archivo descargado.

In [1]:
#modules
from pyspark.sql import  SQLContext, SparkSession
from pyspark import SparkContext

Se inicializa el SparkContext, SQLContext y SparkSession con el fin de inicializar las APIs Spark y Spark SQL.

In [2]:
name = 'exercise 1'
sc = SparkContext(appName= name)
sql_c = SQLContext(sc)

## Pregunta 1 :¿Cuántos registros hay?


Para esto es necesario leer el archivo gracias al método de Spark SQL *read.format* que lee archivos *csv*.

In [3]:
df = sql_c.read.format('csv').options(header='true').load('all_data.csv')
sql_c.registerDataFrameAsTable(df, "all_data")

df.printSchema()
count_output = df.count()

print("shape",(count_output, len(df.columns)))
print("numero de registros:", count_output)

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: string (nullable = true)
 |-- fechaRegistro: string (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)

shape (62530715, 15)
numero de registros: 62530715


Gracias a la función *count()* del DataFrame que generó Spark SQL se obtiene que existen **625,307,155 registros**.

## Pregunta 2 : ¿Cuántas categorías?

El Dataframe tiene una columna de nombre categoría, para identificar todos los valores se  hace una consulta.

In [4]:
df.createOrReplaceTempView("all_data")
sqlDF = sql_c.sql("SELECT DISTINCT categoria FROM all_data")
sqlDF.show(50,False)
count_output = sqlDF.count()
print("shape",(count_output, len(sqlDF.columns)))
print("numero de categorias:", count_output)

+----------------------------------+
|categoria                         |
+----------------------------------+
|MATERIAL ESCOLAR                  |
|ARTS. PARA EL CUIDADO PERSONAL    |
|PESCADOS Y MARISCOS EN CONSERVA   |
|UTENSILIOS DOMESTICOS             |
|categoria                         |
|DETERGENTES Y PRODUCTOS SIMILARES |
|CARNE Y VISCERAS DE RES           |
|PRODUCTOS DE TEMPORADA (NAVIDEÐOS)|
|GALLETAS PASTAS Y HARINAS DE TRIGO|
|HORTALIZAS FRESCAS                |
|null                              |
|DERIVADOS DE LECHE                |
|TORTILLAS Y DERIVADOS DEL MAIZ    |
|GRASAS ANIMALES COMESTIBLES       |
|APARATOS ELECTRONICOS             |
|LEGUMBRES SECAS                   |
|CAFE                              |
|MUEBLES DE COCINA                 |
|CARNES FRIAS SECAS Y EMBUTIDOS    |
|CHOCOLATES Y GOLOSINAS            |
|PAN                               |
|CERVEZA                           |
|MEDICAMENTOS                      |
|CONDIMENTOS                       |
|

La respuesta a la pregunta son **42 categorias**.

## Pregunta 3: ¿Cuántas cadenas comerciales están siendo monitoreadas (y, por lo tanto, reportadas en esa base de datos)?

De la misma forma que se realizó con la pregunta 2 se hace una consulta en la columna de cadenas comerciales.

In [5]:
df.createOrReplaceTempView("all_data")
sqlDF = sql_c.sql("SELECT DISTINCT cadenaComercial FROM all_data")
sqlDF.show(5,False)
count_output = sqlDF.count()
print("shape",(count_output, len(sqlDF.columns)))
print("numero de cadenas comerciales:", count_output)

+------------------------+
|cadenaComercial         |
+------------------------+
|JULIO CEPEDA JUGUETERIAS|
|WOOLWORTH               |
|MARISCOS LA SEPTIMA     |
|PAPELERIA EL TINTERO    |
|LIBRERIA DE SANCHO PANZA|
+------------------------+
only showing top 5 rows

shape (706, 1)
numero de cadenas comerciales: 706


El resultado de la consulta son **706 cadenas comerciales**.

## Pregunta 4 : ¿Cuáles son los productos más monitoreados en cada estado de la república?

Esta pregunta requiere de identificar los productos que más se encuentren por estado, esto se puede realizar gracias al método de *groupby* se hace el conteo de cada valor de producto  con respecto  cada valor de estado.

In [6]:
grouped = df.groupby('producto','estado').count()
grouped.show()

+--------------------+--------------------+-----+
|            producto|              estado|count|
+--------------------+--------------------+-----+
|              FLANAX|    DISTRITO FEDERAL| 6604|
|              PEPINO|          GUANAJUATO| 2280|
|DOLO-NEUROBION FORTE|           QUERÉTARO| 2137|
|             CEBOLLA| MICHOACÁN DE OCAMPO| 3967|
|          SURGAM 300| MICHOACÁN DE OCAMPO|  381|
|             VINAGRE|VERACRUZ DE IGNAC...| 2376|
|        CREMA SOLIDA|VERACRUZ DE IGNAC...| 1738|
|      LECHE EN POLVO|          GUANAJUATO|21360|
|                ADEL|             HIDALGO|  144|
|               DOLAC|          GUANAJUATO| 1749|
|     POLVO P/HORNEAR|             YUCATÁN| 1990|
|CHOCOLATE EN TABL...|              OAXACA| 1621|
|             LENTEJA|              OAXACA| 1719|
|               LIMON|          NUEVO LEÓN| 5387|
|    LECHE CONDENSADA|        QUINTANA ROO| 7040|
|           ZUCARITAS|        QUINTANA ROO| 1290|
|CHOCOLATE EN TABL...|              PUEBLA| 2930|


Con el fin de identificar el producto más monitoreado por estado  se hace a partir de Window donde se especifica el proceso para agrupar los valores de manera descendente.

In [7]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = (Window
          .partitionBy(grouped['estado'])
          .orderBy(grouped['count'].desc(), grouped['producto']))


td = (grouped
 .select('*', rank()
 .over(window)
 .alias('rank')) 
  .filter(col('rank') == 1)
  .orderBy(col('producto'))
  .drop('rank'))

td.show(40,False)

+-----------------------+-------------------------------+------+
|producto               |estado                         |count |
+-----------------------+-------------------------------+------+
|DETERGENTE P/ROPA      |MICHOACÁN DE OCAMPO            |40144 |
|DETERGENTE P/ROPA      |CHIHUAHUA                      |15982 |
|DETERGENTE P/ROPA      |DURANGO                        |9696  |
|DETERGENTE P/ROPA      |ZACATECAS                      |20884 |
|DETERGENTE P/ROPA      |NUEVO LEÓN                     |50307 |
|DETERGENTE P/ROPA      |COLIMA                         |17193 |
|FUD                    |COAHUILA DE ZARAGOZA           |28613 |
|FUD                    |SAN LUIS POTOSÍ                |10164 |
|FUD                    |AGUASCALIENTES                 |12005 |
|FUD                    |QUINTANA ROO                   |34846 |
|FUD                    |CAMPECHE                       |12960 |
|LECHE ULTRAPASTEURIZADA|YUCATÁN                        |35991 |
|LECHE ULTRAPASTEURIZADA|

## Pregunta 5: ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

Para obtener el resultado se realiza una consulta del conteo de los  diferentes productos que cada cadena  comercial tiene en general, como se muestra en el siguiente código donde se indican ***100,754 productos***.


In [8]:
dt = df.groupBy('producto','cadenaComercial').count()
dt.show()

print(dt.count(), "productos por cada cadena comecial")

+--------------------+--------------------+-----+
|            producto|     cadenaComercial|count|
+--------------------+--------------------+-----+
|HORTALIZAS CONGEL...|  COMERCIAL MEXICANA|12927|
|     PAPEL HIGIENICO|  COMERCIAL MEXICANA|17371|
|             BACTRIM|            WAL-MART| 2576|
|             CLEXANE|            WAL-MART| 2999|
|    FUD. TURKEY LINE|             SORIANA| 6763|
|            ASPIRINA|     SANBORN S HNOS.| 3208|
|            RIVOTRIL|         FARMAPRONTO|  167|
|SERVILLETAS DE PA...|            I.M.S.S.| 8655|
|             MOSTAZA|            I.M.S.S.| 5097|
|                ADEL|  FARMACIAS EL FENIX|   94|
|              SANDIA|            CASA LEY|  778|
| HORNO DE MICROONDAS|      BODEGA AURRERA|17435|
|CHOCOLATE EN TABL...|      BODEGA AURRERA|17098|
|      JABON DE PASTA|      BODEGA AURRERA|27554|
|        QUESO OAXACA|MEGA COMERCIAL ME...|  323|
|           ESMERALDA|        SORIANA PLUS| 4441|
|             CERVEZA|              H.E.B.|14959|


Posteriormente, se hace una consulta donde se ordenan las cadenas comerciales por el número de veces que se repiten en la columna.

In [9]:
from pyspark.sql.functions import desc
do = dt.groupBy('cadenaComercial').count().sort(desc("count"))

do.show()
print(do.count(), "centros comerciales")

+--------------------+-----+
|     cadenaComercial|count|
+--------------------+-----+
|             SORIANA| 1059|
|            WAL-MART| 1051|
|MEGA COMERCIAL ME...| 1049|
|  COMERCIAL MEXICANA| 1036|
|            CHEDRAUI| 1026|
|     MERCADO SORIANA| 1024|
|      BODEGA AURRERA| 1012|
|HIPERMERCADO SORIANA| 1006|
|              H.E.B.| 1001|
|        SORIANA PLUS|  999|
|       SORIANA SUPER|  996|
|BODEGA COMERCIAL ...|  979|
|        I.S.S.S.T.E.|  937|
|            SUPERAMA|  936|
|              S MART|  851|
|SUPERMERCADOS SAN...|  849|
|              SUMESA|  848|
|         CITY MARKET|  844|
|FARMACIA GUADALAJARA|  819|
|            CASA LEY|  808|
+--------------------+-----+
only showing top 20 rows

706 centros comerciales


La cadena comercial con mayores productos es ***SORIANA con 1059 productos ***.